# Generating Isochrones

This notebook demonstrates how to generate isochrones from a point using the ORS API, and how to read this data in using geopandas in a useful format.

## Step 1: Have a suitable file containing the locations and coordinates of the locations you wish to create isochrones for
This example will use 'places.gpkg' which contains some locations around Sheffield. It is created manually and saved as a geopackage (.gpkg) file, but you can read in a csv file using geopandas and setting the geometry column. \
You should then run gdf.apply(shapely.wkt.loads) to convert the geometry column to a shapely geometry object.

In [3]:
import geopandas as gpd

places = gpd.GeoDataFrame(columns=['place', 'geometry']).set_geometry('geometry')

In [5]:
from shapely.geometry import Point

places['place'] = ['Hicks Building', 'Town Hall', 'Cathedral']
places['geometry'] = [Point(-1.4864929, 53.3812259), Point(-1.4722871, 53.3805888), Point(-1.4693460, 53.3831771)]

places.set_crs('epsg:4326', inplace=True)
places.to_file('places.gpkg', driver='GPKG', engine='fiona')

In [6]:
places

,place,geometry
0,Hicks Building,POINT (-1.48649 53.38123)
1,Town Hall,POINT (-1.47229 53.38059)
2,Cathedral,POINT (-1.46935 53.38318)


After obtaining a suitable geodataframe and saving at a geopackage, you can read the data into QGIS \
Layer -> Add Layer -> Add Vector Layer -> Select the file under 'source' -> click 'Add'

![](images/IsochronesQGISExample.png)

Now install the ORS Tools plugin in QGIS \
Plugins -> Manage and Install Plugins -> Search for 'ORS Tools' -> Install Plugin \
Look for the new 'ORS Tools' tab in the top bar of QGIS. (default shortcut is Ctrl+R, but this may be different on your system)

![](images/IsochronesQGISPlugin.png)

Click on the settings wheel next to the 'provider' field and paste your ORS API key into the 'API Key' field. \
\
You can obtain an API key by signing up for a free account on the OpenRouteService website. \
\
![](Images/IsochronesQGISAPIKey.png)
\
Generate the isochrones \
Batch Jobs -> Isochrones From Layer -> Select desired parameters -> Run \
The parameter 'Input Layer ID Field' should be set to the name of the column in your input layer that contains the names of the locations so that it can be merged with the places dataframe \
\
![](images/IsochronesParametersExample.png)

You can then save the output layer as a geopackage file by right clicking on the layer in the 'Layers' panel and selecting 'Export' -> 'Save Features As...' \
The saved file will have a geometry column with the isochrone polygons, and the coordinates of the original points in the 'CENTER_LON' and 'CENTER_LAT' columns. \

In [9]:
isochrones = gpd.read_file('isochrones.gpkg')

In [10]:
isochrones

,CENTER_LON,CENTER_LAT,AA_METERS,AA_MODE,TOTAL_POP,geometry
0,-1.4864656622895907,53.38123290077746,1000,foot-walking,27508,"POLYGON ((-1.50081 53.37849, -1.50029 53.37709..."
1,-1.4721797331394337,53.38037667966202,1000,foot-walking,24151,"POLYGON ((-1.48716 53.37802, -1.48715 53.37795..."
2,-1.4693382,53.3830177,1000,foot-walking,20865,"POLYGON ((-1.48472 53.38114, -1.4847 53.38105,..."


This isochrones dataframe can now be used to generate routes from the isochrones to the original points. \
Please refer to the 'Generating routes' notebook for more information on how to do this. \